In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("../Datas/tech_sort1k.csv") #DATA PATH

manual_tag_col = 'exact_matched_patt_contextual' #ENTER MANUAL TAGS COLUMN NAME
text_col = 'summaries'  #ENTER PREDICTION COLUMN NAME

In [3]:
# nltk.download("punkt") #if required then download
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

'''MODIFY IF REQUIRED'''
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text

In [4]:
def give_tags(manual_tags): #give a list of strings then returns all tags as single & multi words
    tech_dict = {}
    for j in manual_tags:
        if j not in tech_dict:
            tech_dict[j] = 1
        else:
            tech_dict[j] += 1

    tech_list=[]
    tech_multi_words=[]
    for i in tech_dict:
        if ((' ' in i) == True):
            tech_multi_words.append(i)
        else:
            tech_list.append(i)
#     if(len(tech_multi_words)>0):
#         print(tech_multi_words)
    return tech_list,tech_multi_words

In [5]:
data[manual_tag_col] = [ [] if x is np.NaN else x for x in data[manual_tag_col] ]
data[manual_tag_col]=data[manual_tag_col].astype(str)
data[manual_tag_col] = data[manual_tag_col].apply(eval) #to convert string to list of strings

data[text_col]=data[text_col].apply(clean_text)
data[text_col] = data[text_col].replace('\s+', ' ', regex=True)

In [7]:
def find_all(a_str, sub):
    '''
    To Get all occurences(index) of a string
    '''
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

def convert(lst):
    #break words
    return ' '.join(lst).split()

def BIO_conversion(sentence,manual_tags):
    '''
    First get Single and Multi-words tags
    Searching for all occurences of the Multi-words tags and storing those indexes 
    '''
    tech_list,tech_multi_words = give_tags(manual_tags)
    ind_dict = {}
    for i in tech_multi_words:
        if i in sentence:
            #ind_dict[sentence.index(i)]=i #
            all_occ = list(find_all(sentence,i)) # [0, 5, 10, 15]
            for ind in all_occ:
                ind_dict[ind]=i #word as value, key as index

    temp_word=""
    final_word_sen=[]
    final_tag_sen = []
    start=-1
    end=-1
    for i in range(0, len(sentence)):
        if(i in range(start,end)): #if the index right now is within a multi word TAG, we dont need to check it
            continue
        if sentence[i].isspace()==True and len(temp_word)>0: #on encountering a SPACE push to word
            final_word_sen.append(temp_word)
            if temp_word in tech_list: #checking if word is in a single word TAG
                final_tag_sen.append("B")
            else:                      #since word is not a Single word TAG, give it O (OUSTIDE) tag
                final_tag_sen.append("O") 
            temp_word="" #resetting the word
        else: #not space
            try: #do we have multiword at this index
                temp_word=ind_dict[i] #GET THE MULTI WORD TAG
                start=i+1              #storing the starting index
                end=i+len(temp_word) #here tag these multiwords and update new i
                listofwords = convert([temp_word]) #get those words splitted
                f=True
                for word in listofwords:
                    final_word_sen.append(word) #pushing the word
                    if f:
                        final_tag_sen.append("B") #Starting of Multi WORD (appending the TAG)
                        f=False                   #MARK FLAG as false, rest of the multi words are I(INSIDE TAG)
                    else: 
                        final_tag_sen.append("I") # appending the TAG
                temp_word=""
            except: #not a space and not a multiword TAG, so just concatenate the string
                if(sentence[i]!=' '):
                    temp_word+=sentence[i]
    return pd.DataFrame(list(zip(final_word_sen, final_tag_sen)),columns =['Word', 'Tag'])

In [9]:
final_data=pd.DataFrame()
for i in range(0,len(data)):
    temp = BIO_conversion(data[text_col][i],data[manual_tag_col][i])
    length = len(temp)
    wordd="Sentence :"+str(i+1) #sentence no.
    a=[wordd]*length
    temp.insert(0,"Sentence #",a)
    final_data = final_data.append(temp, ignore_index=True) #appending sentences in the required format

In [12]:
df=final_data
df.loc[(df['Tag'] == 'B'), 'Tag'] = 'B-ORG'
df.loc[(df['Tag'] == 'I'), 'Tag'] = 'I-ORG'
df.to_csv("output.csv",index=False)